In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import torch
print("torch:", torch.__version__, 
      "cuda:", torch.version.cuda, 
      "is_available:", 
      torch.cuda.is_available())

torch: 2.6.0+cu124 cuda: 12.4 is_available: True


In [10]:
import os 
import requests
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

True

## Inference API

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hf-inference",
    api_key=os.environ["HF_TOKEN"],
)

result = client.translation(
    "Меня зовут Вольфганг и я живу в Берлине",
    model="Helsinki-NLP/opus-mt-ko-en",
)

## pipeline()
task: 텍스트 기반 감성분석, 분류(제로샷), 생성

In [14]:
from transformers import pipeline
classifier = pipeline("text-classification") #task지정
classifier("I want to learn using the refrigerator")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9935249090194702}]

## fine tuning

In [16]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", "tabularisai/multilingual-sentiment-analysis") #task지정
classifier("냉장고를 이용하여 공부하는 것은 즐겁다")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


[{'label': 'Negative', 'score': 0.3945055902004242}]

In [25]:
from transformers import pipeline
classifier = pipeline("fill-mask", "bert-base-multilingual-cased") #task지정
classifier(" 사람의 다리는 [MASK]개 입니다")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'score': 0.12797927856445312,
  'token': 9954,
  'token_str': '한',
  'sequence': '사람의 다리는 한 개 입니다'},
 {'score': 0.12552212178707123,
  'token': 9102,
  'token_str': '두',
  'sequence': '사람의 다리는 두 개 입니다'},
 {'score': 0.08104781061410904,
  'token': 9435,
  'token_str': '세',
  'sequence': '사람의 다리는 세 개 입니다'},
 {'score': 0.036409102380275726,
  'token': 8857,
  'token_str': '개',
  'sequence': '사람의 다리는 개 개 입니다'},
 {'score': 0.027031593024730682,
  'token': 124,
  'token_str': '3',
  'sequence': '사람의 다리는 3 개 입니다'}]

In [ ]:
# question-answering
from transformers import pipeline
question_answerer = pipeline("question-answering", "distilbert-base-cased-distilled-squad")  # 질의 응답
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

In [ ]:
# "ner" 개체명 인식

from transformers import pipeline
ner = pipeline("ner", "dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)  # 개체명 인식
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
# summarization 문장요약

summ=pipeline("summarization", "eenzeenee/t5-base-korean-summarization")
summ(""" 안녕하세요? 우리 (2학년)/(이 학년) 친구들 우리 친구들 학교에 가서 진짜 (2학년)/(이 학년) 이 되고 싶었는데 학교에 못 가고 있어서 답답하죠? 
    그래도 우리 친구들의 안전과 건강이 최우선이니까요 오늘부터 선생님이랑 매일 매일 국어 여행을 떠나보도록 해요. 
    어/ 시간이 벌써 이렇게 됐나요? 늦었어요. 늦었어요. 빨리 국어 여행을 떠나야 돼요. 
    그런데 어/ 국어여행을 떠나기 전에 우리가 준비물을 챙겨야 되겠죠? 국어 여행을 떠날 준비물, 교안을 어떻게 받을 수 있는지 선생님이 설명을 해줄게요. 
    (EBS)/(이비에스) 초등을 검색해서 들어가면요 첫화면이 이렇게 나와요. 
    자/ 그러면요 여기 (X)/(엑스) 눌러주(고요)/(구요). 저기 (동그라미)/(똥그라미) (EBS)/(이비에스) (2주)/(이 주) 라이브특강이라고 되어있죠? 
    거기를 바로 가기를 누릅니다. 자/ (누르면요)/(눌르면요). 어떻게 되냐? b/ 밑으로 내려요 내려요 내려요 쭉 내려요. 
    우리 몇 학년이죠? 아/ (2학년)/(이 학년) 이죠 (2학년)/(이 학년)의 무슨 과목? 국어. 
    이번주는 (1주)/(일 주) 차니까요 여기 교안. 다음주는 여기서 다운을 받으면 돼요. 
    이 교안을 클릭을 하면, 짜잔/. 이렇게 교재가 나옵니다 .이 교안을 (다운)/(따운)받아서 우리 국어여행을 떠날 수가 있어요. 
    그럼 우리 진짜로 국어 여행을 한번 떠나보도록 해요? 국어여행 출발. 자/ (1단원)/(일 단원) 제목이 뭔가요? 한번 찾아봐요. 
    시를 즐겨요 에요. 그냥 시를 읽어요 가 아니에요. 시를 즐겨야 돼요 즐겨야 돼. 어떻게 즐길까? 일단은 내내 시를 즐기는 방법에 대해서 공부를 할 건데요. 
    그럼 오늘은요 어떻게 즐길까요? 오늘 공부할 내용은요 시를 여러 가지 방법으로 읽기를 공부할겁니다. 
    어떻게 여러가지 방법으로 읽을까 우리 공부해 보도록 해요. 오늘의 시 나와라 짜잔/! 시가 나왔습니다 시의 제목이 뭔가요? 다툰 날이에요 다툰 날. 
    누구랑 다퉜나 동생이랑 다퉜나 언니랑 친구랑? 누구랑 다퉜는지 선생님이 시를 읽어 줄 테니까 한번 생각을 해보도록 해요.""")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [ ]:
# image to text
imagetotext = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
imagetotext("https://huggingface.co/Salesforce/blip-image-captioning-large")

## fine tuning을 위한 AutoClass 사용

In [29]:
from datasets import load_dataset
dataset=load_dataset('imdb')
dataset

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 617277.82 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [36]:
# 토크나이저-모델
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
 
tokenized_datasets=dataset.map(preprocess_function, batched=True)
tokenized_datasets    

Map: 100%|██████████| 25000/25000 [00:05<00:00, 4572.82 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [38]:
# 모델 설정
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# 학습 설정
from transformers import TrainingArguments,Trainer

tr_args=TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01
)

trainer=Trainer(
    model=model,
    args=tr_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

In [40]:
trainer.train()

Step,Training Loss
500,0.318000
1000,0.250100
1500,0.227800
2000,0.164500
2500,0.151600
3000,0.154100
3500,0.104200
4000,0.085800
4500,0.093700
5000,0.066100


TrainOutput(global_step=7815, training_loss=0.11724951836213193, metrics={'train_runtime': 3306.1231, 'train_samples_per_second': 37.809, 'train_steps_per_second': 2.364, 'total_flos': 1.6558424832e+16, 'train_loss': 0.11724951836213193, 'epoch': 5.0})

In [41]:
#평가
trainer.evaluate()
#저장
model_path='./fine_tuned_distilbert_imdb'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./fine_tuned_distilbert_imdb\\tokenizer_config.json',
 './fine_tuned_distilbert_imdb\\special_tokens_map.json',
 './fine_tuned_distilbert_imdb\\vocab.txt',
 './fine_tuned_distilbert_imdb\\added_tokens.json',
 './fine_tuned_distilbert_imdb\\tokenizer.json')

In [ ]:
# 저장모델 로드
# 토크나이저로드
# 모델로드
# 모델.eval()



In [ ]:
# 저장 모델로 추론하기
input_text='This movie was absolutely fantastic!'

inputs=tokenizer(input_text, return_tensors='pt') #pytorch 형식으로 텐서를 리턴
inputs

In [ ]:
import torch
with torch.no_grad(): #역전파x, 추론만
    outputs=model(**inputs)
    
predict_class_id=torch.argmax(outputs.logits, dim=-1).item()
print(f'예측 결과: {predict_class_id}')